##### 【 UCI Adult Income Dataset 】<hr>

- 데이터 개요
    * 특성: 수치형 + 범주형 혼합
    * 크기: 약 4만 행 (샘플링 가능)
    * 파일: adult.csv

- 모델 목표
    * 개인의 인구통계/직업 정보를 바탕으로 연소득이 50,000달러 초과인지 여부 예측

- 컬럼 설명
    * class 연소득이 50K 초과(>50K)인지, 이하(<=50K)인지를 나타내는 타깃 레이블
    * age	나이
    * sex	성별 (Male / Female)
    * education-num	교육 수준을 숫자로 표현한 값
    * workclass	고용 형태 (Private, Self-emp, Government 등)
    * occupation	직업 종류
    * hours-per-week	주당 근무 시간
    * capital-gain	자본 이득 (주식, 투자 수익 등)
    * capital-loss	자본 손실
    * marital-status	결혼 상태
    * relationship	가구 내 관계 (Husband, Wife, Not-in-family 등)



### 평가 기준
- 전체 모델 학습 프로세스 적용 여부가 중요함!


[0] 필요한 컬럼 선정 <hr>
- 인구 통계와 직업 정보를 기반 
    - -> target은 class 컬럼
    - -> native_country는 모두 미국으로 추정돼서 삭제
    - -> hours-per-week: 주당 근무 시간은 근무시간이 늘면 당연히 소득도 올라간다고 생각 -> 결과를 학습하는 것이 아닌가? 삭제하기로 결정
    - -> race : 인종 차별적 요소 존재해서 삭제
    - -> fnlwgt : 무슨 컬럼인지 몰라서 삭제
    - -> education-num : 각각 컬럼 값들의 순서를 몰라서 삭제
    - -> education : 설명에 없어서 삭제

[1] 모듈 및 데이터 로딩 <hr>

In [12]:
# ==================================================================
# [1-1] 모듈 로딩
# ==================================================================

# 전처리 
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# ML 학습
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

# ML 모델
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import all_estimators

# ML 전처리
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# ML 성능평가
from sklearn.metrics import accuracy_score, classification_report


# ==================================================================
# [1-2] 데이터 준비 
# ==================================================================

FILE_NAME = './adult.csv'
dataDF = pd.read_csv(FILE_NAME)
dataDF.drop(['native-country', 'Unnamed: 0', 'hours-per-week', 'fnlwgt', 'race', 'education-num', 'education'], inplace=True, axis=1)
display(dataDF.head())

,age,workclass,marital-status,occupation,relationship,sex,capital-gain,capital-loss,class
0,25,Private,Never-married,Machine-op-inspct,Own-child,Male,0,0,<=50K
1,38,Private,Married-civ-spouse,Farming-fishing,Husband,Male,0,0,<=50K
2,28,Local-gov,Married-civ-spouse,Protective-serv,Husband,Male,0,0,>50K
3,44,Private,Married-civ-spouse,Machine-op-inspct,Husband,Male,7688,0,>50K
4,18,NaN,Never-married,NaN,Own-child,Female,0,0,<=50K


[2] 데이터 확인 및 전처리 <hr>

In [13]:
# ==================================================================
# [2-1] 데이터 확인
# ==================================================================
print(dataDF.info(), '\n')
print(dataDF.describe())

# 상관계수
numeric_columns = dataDF.select_dtypes(include=[np.number]).columns
print(dataDF[numeric_columns].corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   marital-status  48842 non-null  object
 3   occupation      46033 non-null  object
 4   relationship    48842 non-null  object
 5   sex             48842 non-null  object
 6   capital-gain    48842 non-null  int64 
 7   capital-loss    48842 non-null  int64 
 8   class           48842 non-null  object
dtypes: int64(3), object(6)
memory usage: 3.4+ MB
None 

                age  capital-gain  capital-loss
count  48842.000000  48842.000000  48842.000000
mean      38.643585   1079.067626     87.502314
std       13.710510   7452.019058    403.004552
min       17.000000      0.000000      0.000000
25%       28.000000      0.000000      0.000000
50%       37.000000      0.000000      0.000000
75%       48.00

In [14]:
# ==================================================================
# [2-2] 결측치, 중복값, 이상치 처리
# ==================================================================
# 결측치 확인
print(dataDF.isnull().sum())        # 결측치 존재
                                    # workclass : 고용형태 - 없는 건 기타로 넣기
                                    # occupation : 직업종류 - 없는 건 기타로 넣기

# 중복값 확인
print(dataDF.duplicated().sum())    # 중복값 0

# 이상치 확인(IQR 기반)
for col in numeric_columns:
    Q1 = dataDF[col].quantile(0.25)
    Q3 = dataDF[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = dataDF[(dataDF[col] < lower) | (dataDF[col] > upper)]
    print(f"{col}: {len(outliers)}개")

    # capital-gain, capital-loss는 연소득을 강하게 나타내는 케이스라서, 제거하지 않기로 함
    # adult, education_num은 그럴 수 있다고 생각함 -> 제거X

age                  0
workclass         2799
marital-status       0
occupation        2809
relationship         0
sex                  0
capital-gain         0
capital-loss         0
class                0
dtype: int64
28694
age: 216개
capital-gain: 4035개
capital-loss: 2282개


[3] 피쳐와 타겟 분리 <hr>

In [15]:
# 타겟
dataDF["class"] = dataDF["class"].map({">50K": 1, "<=50K": 0})  # 이상 : 1, 이하 : 0
targetSR = dataDF["class"]
print(f"targetSR  : {targetSR.shape}")

# 피쳐
featureDF = dataDF.iloc[:, :-1]
print(f"featureDF : {featureDF.shape}")

targetSR  : (48842,)
featureDF : (48842, 8)


[4] 수치형, 범주형 인코딩/전처리 <hr>

In [16]:
# 수치형 컬럼
numeric_columns = featureDF.select_dtypes(include=[np.number]).columns.tolist()

# 결측치를 'Others'
categorical_features_others = ["workclass", "occupation"]

# 나머지 범주형
categorical_features = ['marital-status', 'relationship']

# 수치형 변환기
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# 범주형 변환기1 ('Others'로 채움)
onehot_transformer_others = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Others")),
    ("onehot", OneHotEncoder())
])

# 범주형 변환기2 
onehot_transformer_mode = Pipeline(steps=[
    ("onehot", OneHotEncoder())
])

# 전체 전처리 결합
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_columns),
        ("ohe_others", onehot_transformer_others, categorical_features_others),
        ("ohe_mode", onehot_transformer_mode, categorical_features)
    ],
    remainder="drop"
)


[4] 훈련용, 테스트용 데이터셋 분리 <hr>


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    featureDF,
    targetSR,
    test_size=0.2,
    stratify=targetSR
)
print(f"train : {x_train.shape}")
print(f"test  : {x_test.shape}")

train : (39073, 8)
test  : (9769, 8)


[5] all_estimators <hr>

In [18]:
# ==================================================================
# [5-1] 모든 분류 모델 가져오기
# ==================================================================

estimators = all_estimators(type_filter="classifier")
print(len(estimators))

# ==================================================================
# [5-2] 각 모델 테스트
# ==================================================================

results = []

for name, estimatorClass in estimators:
    try:

        model = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("classifier", estimatorClass())
        ])
        
        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        results.append({
            'model': name,
            'accuracy': accuracy,
            'class': estimatorClass
        })
        print(f"{name}: {accuracy}")

    except Exception:
        continue

# ==================================================================
# [5-3] 결과 
# ==================================================================

resultDF = pd.DataFrame(results).sort_values('accuracy', ascending=False)
print("\n=== 모델 성능 순위 ===")
display(resultDF[['model', 'accuracy']].head(10))

# 상위 1 bestModel
bestModelClass = resultDF.iloc[0]['class']
print("\n=== Best Model Class ===")
print(bestModelClass)

44
AdaBoostClassifier: 0.8446105026102979
BaggingClassifier: 0.8446105026102979
BernoulliNB: 0.7545296345582967
CalibratedClassifierCV: 0.8370355205241069
DecisionTreeClassifier: 0.8431773978912888
DummyClassifier: 0.7606715119254785
ExtraTreeClassifier: 0.8318149247620023


KeyboardInterrupt: 

[7] Pipeline 구축 <hr>

In [ ]:
finalPipeline = Pipeline([
    ('scaler', StandardScaler()),
    ("preprocessor", preprocessor),
    ('model' , bestModelClass())
])

bestModel = finalPipeline.fit(x_train, y_train)

bestModel.score(x_test, y_test)
print(classification_report(y_test, bestModel.predict(x_test)))

ValueError: could not convert string to float: 'Private'